In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

print(f"OPENAI_API_KEY is set to: {OPENAI_API_KEY is not None}")

OPENAI_API_KEY is set to: True


### State

In [3]:
from typing import Annotated

from typing_extensions import TypedDict

from langgraph.graph.message import AnyMessage, add_messages


class State(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]

### Agent

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

def answer_conversation(state: State):

    all_messages = state["messages"]
    last_message = state["messages"][-1]

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    chat_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a seasoned study and work counselor."
             " Provide thoughtful and empathetic advice based on the user's messages and guide"
             " them through finding a suitable study or work career."
             " Answer the last message based on your knowledge of all previous messages."),
            ("user", "Last message: {last_message}. All previous messages: {all_messages}"),
        ]
    )

    response = llm.predict_messages(
        chat_prompt.format_messages(
            last_message=last_message,
            all_messages=all_messages
        )
    )

    return {"messages": [response]}

### Graph

In [11]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import END, StateGraph, START
from langchain_core.runnables.graph import MermaidDrawMethod 
from langgraph.prebuilt import tools_condition
import uuid

builder = StateGraph(State)
builder.add_node("answer_conversation", answer_conversation)
builder.add_edge(START, "answer_conversation")
builder.add_edge("answer_conversation", END)

simple_chat = builder.compile(checkpointer=InMemorySaver())

thread_id = str(uuid.uuid4())
config = {"configurable": {"thread_id": thread_id}}

In [6]:
# from IPython.display import Image, display

# graph_obj = simple_chat.get_graph()

# # Try different rendering methods to avoid the asyncio event loop issue
# try:
#     # First try without Pyppeteer (uses mermaid.ink API)
#     graph_png = graph_obj.draw_mermaid_png(
#         output_file_path="langgraph_example.png",
#         max_retries=5,
#         retry_delay=2.0,
#     )
#     display(Image(graph_png))
# except Exception as e:
#     print(f"Failed to render with mermaid.ink: {e}")
#     try:
#         # Alternative: render as ASCII
#         print("\nGraph structure (ASCII):")
#         print(graph_obj.draw_ascii())
#     except Exception as e2:
#         print(f"Failed to render ASCII: {e2}")
#         # Fallback: show the graph structure programmatically
#         print("\nGraph nodes and edges:")
#         print(f"Nodes: {list(graph_obj.nodes.keys())}")
#         print(f"Edges: {list(graph_obj.edges)}")

In [20]:
def talk_to_model(input_text: str):
    response = simple_chat.invoke({
        "messages": input_text
    }, config=config)

    print(response["messages"][-1].content)
    return response

conversation = talk_to_model("I am interested in being in the nature and take care of animals.")


It’s wonderful to hear about your interest in nature and caring for animals! This passion can lead you to a fulfilling career that aligns with your values and interests. Here are a few paths you might consider:

1. **Wildlife Conservation**: This field involves protecting and managing wildlife and their habitats. You could work with organizations focused on conservation efforts, conducting research, or even engaging in fieldwork.

2. **Veterinary Medicine**: If you love animals and want to provide direct care, becoming a veterinarian or a veterinary technician could be a great fit. This path requires specific education and training, but it can be incredibly rewarding.

3. **Animal Care and Training**: Working in animal shelters, zoos, or as a dog trainer can allow you to work closely with animals. You could also consider roles in animal rehabilitation or rescue organizations.

4. **Environmental Science**: If you’re interested in the broader aspects of nature, studying environmental sc

In [21]:
conversation = talk_to_model("Hm, I would like to add that I'm also very technical and I like to work with computers.")

It's great to hear that you have a strong technical inclination and enjoy working with computers! This opens up a wide range of career paths for you. Given your interests, you might want to consider fields such as:

1. **Information Technology (IT)**: This could involve roles in network administration, system analysis, or IT support. These positions often require problem-solving skills and a good understanding of computer systems.

2. **Software Development**: If you enjoy coding, you might explore becoming a software developer or programmer. This field allows you to create applications, websites, or software solutions, and it can be very rewarding.

3. **Data Science or Data Analysis**: If you have an interest in working with data, this could be a great fit. Data scientists and analysts use technical skills to interpret complex data and help organizations make informed decisions.

4. **Cybersecurity**: With the increasing importance of data protection, a career in cybersecurity could 

In [23]:
conversation = talk_to_model("What was the first message I sent you?")

It seems like you're looking for a recap of our previous conversation, but I don't have access to past messages. However, I'm here to help you with your study or career path! If you could share a bit about your interests, skills, or any specific fields you're considering, I can provide tailored advice to guide you in finding a suitable study or work career. What are you passionate about?


In [23]:
# Different ways to access the results from simple_chat.invoke()

# 1. Store the result in a variable for easy access
result = simple_chat.invoke({
    "messages": "What skills do I need to develop for wildlife biology?"
}, config=config)

print("=== Full Result Structure ===")
print(f"Type: {type(result)}")
print(f"Keys: {result.keys()}")
print()

print("=== All Messages ===")
for i, message in enumerate(result["messages"]):
    print(f"Message {i+1}: {type(message).__name__}")
    print(f"Content: {message.content[:100]}...")
    print()

print("=== Just the AI Response ===")
# The AI response is typically the last message
ai_response = result["messages"][-1]
print(f"AI Response Type: {type(ai_response).__name__}")
print(f"AI Response Content:")
print(ai_response.content)

=== Full Result Structure ===
Type: <class 'dict'>
Keys: dict_keys(['messages'])

=== All Messages ===
Message 1: HumanMessage
Content: I am interested in being in the nature and take care of animals....

Message 2: HumanMessage
Content: It's wonderful to hear about your passion for nature and caring for animals! There are many fulfilli...

Message 3: HumanMessage
Content: What skills do I need to develop for wildlife biology?...

Message 4: HumanMessage
Content: It's great to see your enthusiasm for pursuing a career in wildlife biology! This field is not only ...

=== Just the AI Response ===
AI Response Type: HumanMessage
AI Response Content:
It's great to see your enthusiasm for pursuing a career in wildlife biology! This field is not only rewarding but also crucial for the conservation of our planet's biodiversity. To excel as a wildlife biologist, there are several key skills and areas of knowledge you should focus on developing:

1. **Scientific Knowledge**: A strong foundation 

## Alternative: Better Conversation History Approach

In [12]:
# Alternative approach: Use the full conversation history more naturally
def answer_conversation_v2(state: State):
    """
    Better version that passes the full conversation to the LLM
    instead of trying to format it in the prompt
    """
    all_messages = state["messages"]
    
    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    
    # Create system message
    system_message = ("You are a seasoned study and work counselor. "
                     "Provide thoughtful and empathetic advice based on the user's messages and guide "
                     "them through finding a suitable study or work career. "
                     "Consider the full conversation history when responding.")
    
    # Build the conversation for the LLM
    conversation_for_llm = [("system", system_message)]
    
    # Add all previous messages to maintain natural conversation flow
    for msg in all_messages:
        if hasattr(msg, 'content'):
            # Determine if it's a user or assistant message
            if 'Human' in msg.__class__.__name__:
                conversation_for_llm.append(("user", msg.content))
            else:
                conversation_for_llm.append(("assistant", msg.content))
    
    chat_prompt = ChatPromptTemplate.from_messages(conversation_for_llm)
    
    response = llm.predict_messages(chat_prompt.format_messages())
    
    return {"messages": [response]}

In [ ]:
# Test the improved conversation history
print("=== Testing Improved Conversation History ===")

# Create a new graph with the improved function (optional)
builder_v2 = StateGraph(State)
builder_v2.add_node("answer_conversation", answer_conversation_v2)
builder_v2.add_edge(START, "answer_conversation")
builder_v2.add_edge("answer_conversation", END)

simple_chat_v2 = builder_v2.compile(checkpointer=InMemorySaver())

# Test with a fresh conversation
test_thread_id = str(uuid.uuid4())
test_config = {"configurable": {"thread_id": test_thread_id}}

def stream_graph_updates(user_input: str):
    # Fix: Pass the user input as a simple string, not as a dict with role/content
    for event in simple_chat_v2.stream({"messages": user_input}, config=test_config):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)

=== Testing Improved Conversation History ===


In [16]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    stream_graph_updates(user_input)

ValueError: Checkpointer requires one or more of the following 'configurable' keys: thread_id, checkpoint_ns, checkpoint_id

In [15]:
print("Starting test conversation...")

# First message
response1 = simple_chat_v2.invoke({
    "messages": "I love working with animals and being outdoors."
}, config=test_config)



print(response1["messages"][-1].content)

Starting test conversation...


C:\Users\MartinHallberg\AppData\Local\Temp\ipykernel_2368\2133682887.py:31: LangChainDeprecationWarning: The method `BaseChatModel.predict_messages` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict_messages(chat_prompt.format_messages())


It's wonderful to hear that you have a passion for working with animals and being outdoors! There are many career paths that could align with your interests. Here are a few options to consider:

1. **Veterinary Technician or Technologist**: If you enjoy caring for animals and have an interest in veterinary medicine, this could be a great fit. It involves assisting veterinarians in clinics or animal hospitals.

2. **Wildlife Biologist or Conservationist**: If you’re interested in studying animals in their natural habitats and working on conservation efforts, this could be a fulfilling career. It often involves fieldwork and research.

3. **Zoologist**: This role involves studying animals in captivity and the wild, focusing on their behavior, physiology, and conservation. It often requires a degree in biology or a related field.

4. **Animal Trainer or Behaviorist**: If you enjoy working directly with animals and helping them learn, this could be a rewarding path. It can involve training

In [ ]:
def chat_with_ai(input_text: str):
    response = simple_chat_v2.invoke({
        "messages": input_text
    }, config=config)

    print(response["messages"][-1].content)
    return response

It's wonderful to hear that you have a passion for working with animals and being outdoors! There are many fulfilling career paths that can align with your interests. Here are a few options you might consider:

1. **Veterinary Technician**: This role allows you to work closely with animals in a clinical setting, assisting veterinarians in providing care. It often involves both hands-on work with animals and some outdoor activities, especially if you work with wildlife or in rural areas.

2. **Wildlife Biologist**: If you enjoy research and conservation, this could be a great fit. Wildlife biologists study animals in their natural habitats, which often involves fieldwork outdoors.

3. **Zoologist**: Similar to wildlife biologists, zoologists study animals, but they often work in zoos or aquariums. This role can involve both research and direct care of animals.

4. **Animal Trainer or Behaviorist**: If you have a knack for understanding animal behavior, you might enjoy training animals f

In [ ]:
# Second message - should remember the first
response2 = simple_chat_v2.invoke({
    "messages": "I'm also very good with technology and programming."
}, config=test_config)

print("🤖 AI:", response2["messages"][-1].content[:100] + "...")

# Third message - test if it remembers both previous messages
response3 = simple_chat_v2.invoke({
    "messages": "What career combines all these interests I mentioned?"
}, config=test_config)

print("🤖 AI:", response3["messages"][-1].content[:200] + "...")

print(f"\n📊 Total messages in conversation: {len(response3['messages'])}")
print("✅ The AI should now reference both your love for animals/outdoors AND your tech skills!")

## Stream Chat